# A Convex Programming example: Geometric Programming (using log-sum-exp)
Suppose we are designing a box of height $h$, width $w$, and depth $d$ with maximum volume $hwd$ subject to the limits:
 * Total wall area: $2hw + 2hw \leq A_{wall}$ 
 * Total floor area: $wd \leq A_{floor}$
 * Height-width aspect ratio: $\alpha \leq h/w \leq \beta$
 * Width-depth aspect ratio: $\gamma \leq w/d \leq \delta$

Suppose we reformulate our geometric program as follows:

\begin{align}
\min_{(𝑥,𝑦,𝑧)}⁡&-x-y-z\\
\text{s.t.}& \ \log⁡(e^{(log⁡(2/𝐴_{𝑤𝑎𝑙𝑙} )+𝑥+𝑦)})+log⁡(e^{(log⁡(2/𝐴_{𝑤𝑎𝑙𝑙} )+𝑥+𝑧)} )\leq0\\
& \ 𝑦+𝑧\leq \log A_{floor}\\
& \ \log⁡ 𝛼 \leq 𝑥−𝑦 \leq \log ⁡𝛽\\
& \ \log⁡ 𝛾\leq 𝑧−𝑦 \leq \log ⁡𝛿
\end{align}

Now suppose we want the wall area to be no greater than 50 $m^2$ and the floor area to be no more than 60 $m^2$. The height-width aspect ratio should be between 10 and 12. The width-depth aspect ratio should be between 0.1 and 0.4. What is the maximum volume box we can build?

In [3]:
### DATA ###

Awall = 50
Afloor = 60
α = 10
β = 12
γ = 0.1
δ = 0.4

### MODEL ###

using JuMP, Ipopt

m = Model(Ipopt.Optimizer)

@variable(m, x)
@variable(m, y)
@variable(m, z)

@objective(m, Min, -x-y-z)

@NLconstraint(m,log(exp(log(2/Awall)+x+y))+log(exp(log(2/Awall)+x+z)) <= 0)
@constraint(m, y+z <= log(Afloor))
@constraint(m, log(α) <= x-y <= log(β))
@constraint(m, log(γ) <= z-y <= log(δ))

optimize!(m)

println()
h = exp(value(x))
w = exp(value(y))
d = exp(value(z))
println("Height: ", h)
println("Width: ", w)
println("Depth: ", d)
println("Max volume: ", h*w*d)


This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:       13
Number of nonzeros in Lagrangian Hessian.............:        5

Total number of variables............................:        3
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        6
        inequality constraints with only lower bounds:        2
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        4

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 2.30e+00 4.98e-01  -1.0 0.00e+00    -  0.00e+00 0.00e+00  